<a href="https://colab.research.google.com/github/WillianXG/Projeto_Em_grupo-Modulo-3/blob/3-pergunta-2/grupoModulo3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dados Criptomoedas

In [ ]:
import pandas as pd
import requests
from matplotlib import pyplot as plt
import seaborn as sns

aave = pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Aave.csv')
binance=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_BinanceCoin.csv')
bitcoin=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Bitcoin.csv')
cardano=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Cardano.csv')
chainlink=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_ChainLink.csv')
cosmos=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Cosmos.csv')
cryptocom=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_CryptocomCoin.csv')
dogecoin=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Dogecoin.csv')
eos=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_EOS.csv')
ethereum=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Ethereum.csv')
iota=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Iota.csv')
litecoin=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Litecoin.csv')
monero=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Monero.csv')
nem=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_NEM.csv')
polkadot=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Polkadot.csv')
solana=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Solana.csv')
stellar=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Stellar.csv')
tether=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Tether.csv')
tron=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Tron.csv')
usd=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_USDCoin.csv')
uniswap=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Uniswap.csv')
wrappedBitcoin=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_WrappedBitcoin.csv')
xrp=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_XRP.csv')
criptomoedas=[aave, binance, bitcoin, cardano, chainlink, cosmos, cryptocom, dogecoin, eos, ethereum, iota, litecoin, monero, nem, polkadot, solana, stellar, tether, tron, usd, uniswap, wrappedBitcoin, xrp]




# Analise de Dados


In [ ]:
criptomoedaMedia=[]
for criptomoeda in criptomoedas:
  criptomoeda['Date'] = pd.to_datetime(criptomoeda['Date'])
  # Corrigindo possíveis
  # 1. Verificar valores ausentes
  # valoresEncontrados = criptomoeda.isnull().sum()
  # Exibindo valores nulos
  # print("\nValores Nulos:\n", valoresEncontrados[valoresEncontrados > 0]) if valoresEncontrados.any() else print("Nenhum valor nulo encontrado.")
  # 2. Verificar erros de tipo de dados na coluna 'Date'
  # tipo = pd.DataFrame(criptomoeda.dtypes)
  #   # Exibindo tipos antes da correção
  # print(tipo)
  # tipo = pd.DataFrame(criptomoeda.dtypes)
  # criptomoeda = criptomoeda.astype({col: 'string' for col in criptomoeda.select_dtypes(include='object').columns})
  # # Exibindo tipos pós correção
  # print('\n\n\n')
  # print(criptomoeda['Name'][0])
  # tipo = pd.DataFrame(criptomoeda.dtypes)
  # print(tipo)
  # print('\n\n\n')
  #3. Verificar duplicatas
  duplicatas = criptomoeda.duplicated().sum()
  print(duplicatas)
  if duplicatas > 0:
       criptomoeda=criptomoeda.drop_duplicates()
       print("\nDuplicate Rows Found: ", duplicatas)
  else: print("Sem Colunas duplicadas.")
  duplicatas = criptomoeda.duplicated().sum()
  print(f'Depois de corrigir \n\n{duplicatas}')
  if duplicatas > 0:
       print("\nDuplicate Rows Found: ", duplicatas)
  else: print("Sem Colunas duplicadas.")

  # # 4. (Opcional) Verificar outliers na coluna 'Close'
  # if 'Close' in criptomoeda.columns:
  #     q1 = criptomoeda['Close'].quantile(0.25)
  #     q3 = criptomoeda['Close'].quantile(0.75)
  #     iqr = q3 - q1
  #     lower_bound = q1 - 1.5 * iqr
  #     upper_bound = q3 + 1.5 * iqr
  #     outliers = criptomoeda[(criptomoeda['Close'] < lower_bound) | (criptomoeda['Close'] > upper_bound)]
  #     if not outliers.empty:
  #         print("\nOutliers Found:\n", outliers[['Date', 'Close']]
  # Corrigir os problemas encontrado
  # Remover linhas onde a data não pôde ser convertida
  #criptomoeda = criptomoeda.dropna(subset=['Date']
  # Preencher valores ausentes

    # Calculando valores das perguntas
        #1. Como se comportaram os valores para todas as criptomoedas? Os valores tiveram uma tendência de queda ou de aumento?
        #2. Quais os valores médios para todas as criptomoedas?
        #3. Em quais anos houve maiores quedas e valorizações?
        #4. Existe alguma tendência de aumento ou queda dos valores pelo dia da semana?
        #5. Qual moeda se mostra mais interessante em relação à valorização pela análise da série histórica?
        #6. Qual moeda se mostra menos interessante em relação à valorização pela análise da série histórica?
        #7. Existe correlação entre os valores para todas as criptomoedas?
        #8.
        #9.
    # Repondendo as perguntas
        #1. Como se comportaram os valores para todas as criptomoedas? Os valores tiveram uma tendência de queda ou de aumento?
        #2. Quais os valores médios para todas as criptomoedas?
print(criptomoedas[1])
        #3. Em quais anos houve maiores quedas e valorizações?
        #4. Existe alguma tendência de aumento ou queda dos valores pelo dia da semana?
        #5. Qual moeda se mostra mais interessante em relação à valorização pela análise da série histórica?
        #6. Qual moeda se mostra menos interessante em relação à valorização pela análise da série histórica?
        #7. Existe correlação entre os valores para todas as criptomoedas?
        #8.
        #9.

0
Sem Colunas duplicadas.
Depois de corrigir 

0
Sem Colunas duplicadas.
0
Sem Colunas duplicadas.
Depois de corrigir 

0
Sem Colunas duplicadas.
0
Sem Colunas duplicadas.
Depois de corrigir 

0
Sem Colunas duplicadas.
0
Sem Colunas duplicadas.
Depois de corrigir 

0
Sem Colunas duplicadas.
0
Sem Colunas duplicadas.
Depois de corrigir 

0
Sem Colunas duplicadas.
0
Sem Colunas duplicadas.
Depois de corrigir 

0
Sem Colunas duplicadas.
0
Sem Colunas duplicadas.
Depois de corrigir 

0
Sem Colunas duplicadas.
0
Sem Colunas duplicadas.
Depois de corrigir 

0
Sem Colunas duplicadas.
0
Sem Colunas duplicadas.
Depois de corrigir 

0
Sem Colunas duplicadas.
0
Sem Colunas duplicadas.
Depois de corrigir 

0
Sem Colunas duplicadas.
0
Sem Colunas duplicadas.
Depois de corrigir 

0
Sem Colunas duplicadas.
0
Sem Colunas duplicadas.
Depois de corrigir 

0
Sem Colunas duplicadas.
0
Sem Colunas duplicadas.
Depois de corrigir 

0
Sem Colunas duplicadas.
0
Sem Colunas duplicadas.
Depois de corrigir 

0
Se

1. Como se comportaram os valores para todas as criptomoedas? Os valores tiveram uma
tendência de queda ou de aumento?
2. Quais os valores médios para todas as criptomoedas?
3. Em quais anos houve maiores quedas e valorizações?
4. Existe alguma tendência de aumento ou queda dos valores pelo dia da semana?
5. Qual moeda se mostra mais interessante em relação à valorização pela análise da série
histórica?
6. Qual moeda se mostra menos interessante em relação à valorização pela análise da série
histórica?
7. Existe correlação entre os valores para todas as criptomoedas?

2.   Quais os valores médios para todas as criptomoedas? *VALDEILSON*



In [ ]:
# JUNTANDO TODOS OS CSVs EM APNES UM.


#listando o novo csv
csv_unico  = [
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Aave.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_BinanceCoin.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Bitcoin.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Cardano.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_ChainLink.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Cosmos.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_CryptocomCoin.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Dogecoin.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_EOS.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Ethereum.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Iota.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Litecoin.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Monero.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_NEM.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Polkadot.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Solana.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Stellar.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Tether.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Tron.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_USDCoin.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Uniswap.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_WrappedBitcoin.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_XRP.csv'
]


li = []  #lista para armazenar o arquivo csv novo


# Gerando um DataFrame com uma função em loop

for url in csv_unico:
    df = pd.read_csv(url)
    li.append(df)

df_unico = pd.concat(li, axis=0, ignore_index=True)    # Realizando a concatenização
df_unico



# prompt: Quais os valores médios para todas as criptomoedas?

df_unico['Date'] = pd.to_datetime(df_unico['Date'])
df_unico = df_unico[(df_unico['Date'] >= '2020-01-01') & (df_unico['Date'] <= '2021-01-01')]

media_por_moeda = df_unico.groupby('Name')['Close'].mean()
media_por_moeda

Name
Aave                  61.664022
Binance Coin          21.350415
Bitcoin            11116.378080
Cardano                0.086795
Chainlink              7.428178
Cosmos                 4.126263
Crypto.com Coin        0.092467
Dogecoin               0.002743
EOS                    2.913525
Ethereum             307.542978
IOTA                   0.251735
Litecoin              56.243496
Monero                84.943611
NEM                    0.083275
Polkadot               4.822105
Solana                 1.700169
Stellar                0.082374
TRON                   0.020719
Tether                 1.001316
USD Coin               1.002074
Uniswap                3.508140
Wrapped Bitcoin    11134.258119
XRP                    0.258399
Name: Close, dtype: float64

# Pergunta 3

## Em quais anos houve maiores quedas e valorizações?

In [ ]:
def data_process(df):
    df['Year'] = df['Date'].dt.year
    return df

criptomoedas = [data_process(df) for df in criptomoedas]


def calculate_annual(df):
    df_grouped = df.groupby('Year')['Close'].agg(['first', 'last'])
    df_grouped['Change'] = (df_grouped['last'] - df_grouped['first']) / df_grouped['first']
    df_grouped = df_grouped.rename(columns={'first': 'Primeira', 'last': 'Última', 'Change': 'Diferença'})
    return df_grouped

annual_changes = [calculate_annual(df) for df in criptomoedas]

def find_extremes(df):
    max_val_year = df['Diferença'].idxmax()
    min_val_year = df['Diferença'].idxmin()
    return max_val_year, df.loc[max_val_year, 'Diferença'], min_val_year, df.loc[min_val_year, 'Diferença']

extremes = [find_extremes(df) for df in annual_changes]

crypto_names = ['Aave', 'BinanceCoin', 'Bitcoin', 'Cardano', 'ChainLink', 'Cosmos', 'CryptocomCoin',
                'Dogecoin', 'EOS', 'Ethereum', 'Iota', 'Litecoin', 'Monero', 'NEM', 'Polkadot',
                'Solana', 'Stellar', 'Tether', 'Tron', 'USDCoin', 'Uniswap', 'WrappedBitcoin', 'XRP']

result_dataframes = []

for i, crypto in enumerate(crypto_names):
    max_year, max_change, min_year, min_change = extremes[i]
    df = pd.DataFrame({
        'Crypto': [crypto],
        'Melhor Ano': [max_year],
        'Maior valorização': [max_change],
        'Pior Ano': [min_year],
        'Menor valorizaçao': [min_change]
    })
    result_dataframes.append(df)

for i in result_dataframes:
    df = pd.concat(result_dataframes, ignore_index=True)
    df = pd.DataFrame(df)
df


,Crypto,Melhor Ano,Maior valorização,Pior Ano,Menor valorizaçao
0,Aave,2021,2.507467,2020,0.644659
1,BinanceCoin,2017,81.135673,2018,-0.267378
2,Bitcoin,2017,13.180152,2018,-0.725954
3,Cardano,2017,26.753375,2018,-0.943646
4,ChainLink,2020,5.238396,2018,-0.605427
5,Cosmos,2021,1.242353,2019,-0.438097
6,CryptocomCoin,2021,1.061801,2018,0.181833
7,Dogecoin,2021,40.235428,2018,-0.736714
8,EOS,2017,2.234608,2018,-0.709728
9,Ethereum,2017,91.594245,2018,-0.827387
